# Generate a sample JSON : 

### Imports and Variables : 

In [36]:
import json
import random
from faker import Faker
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display
import os

fake = Faker()

### Upload Sample JSON

In [ ]:
upload = widgets.FileUpload(accept=".json", multiple=False)
display(widgets.HTML("<b>Upload a sample JSON file:</b>"))
display(upload)


### Input Widget

In [38]:
supplier_input = widgets.Text(description='SupplierID:', placeholder='Enter SupplierID')
count_input = widgets.IntText(description='Count:', value=50)
generate_btn = widgets.Button(description="Generate Data", button_style="success")

output_box = widgets.Output()

### Generate fake values based on json inputed

In [39]:
def infer_generator(key, value):
    """Infer a generator based on value type."""
    if key.lower() == "supplierid":
        return None  # will be set manually
    if isinstance(value, str):
        if "@" in value:
            return lambda: fake.email()
        elif value.isdigit():
            return lambda: fake.random_number(digits=len(value))
        elif "-" in value and ":" in value:
            return lambda: fake.date_time_between(start_date="-10y", end_date="now").isoformat() + "Z"
        elif len(value) < 10:
            return lambda: fake.word().upper()
        else:
            return lambda: fake.text(max_nb_chars=len(value))
    elif isinstance(value, float):
        return lambda: round(random.uniform(0.5 * value, 1.5 * value), 2)
    elif isinstance(value, int):
        return lambda: random.randint(int(0.5 * value), int(1.5 * value))
    elif isinstance(value, bool):
        return lambda: random.choice([True, False])
    else:
        return lambda: None

def generate_data(sample_obj, count, supplier_id):
    def recursive_generate(sample):
        if isinstance(sample, dict):
            result = {}
            for key, val in sample.items():
                if key == "SupplierID":
                    result[key] = supplier_id
                elif isinstance(val, dict):
                    result[key] = recursive_generate(val)
                elif isinstance(val, list):
                    if val and isinstance(val[0], dict):
                        # Generate random number of items (e.g. 1 to 5) per list
                        result[key] = [recursive_generate(item) for item in val]
                    else:
                        result[key] = val  # Leave as is if not list of dicts
                else:
                    gen = infer_generator(key, val)
                    result[key] = gen() if gen else val
            return result
        return sample

    return [recursive_generate(sample_obj) for _ in range(count)]





### Generate Data

In [40]:
def get_upload_file_content(upload_widget):
    val = upload_widget.value
    with output_box:
        print("Inside get_upload_file_content, value type:", type(val))
        print("Value content preview:", str(val)[:500])
    if not val:
        return None

    try:
        # ipywidgets >= 8 returns a dict; older versions may return a tuple
        upload_item = next(iter(val.values())) if isinstance(val, dict) else val[0]
        mem_view = upload_item.get('content')
        if mem_view:
            return mem_view.tobytes()
    except Exception as e:
        with output_box:
            print("❌ Error accessing uploaded file content:", e)

    with output_box:
        print("Unknown upload.value format")
    return None


### On Click Generate 


In [41]:
def on_generate_clicked(b):
    output_box.clear_output()

    if not upload.value:
        with output_box:
            print("❌ Please upload a sample JSON file first.")
        return

    try:
        raw = get_upload_file_content(upload)
        text = raw.decode('utf-8')
        try:
            sample = json.loads(text)
        except json.JSONDecodeError as e:
            with output_box:
                print("Standard JSON failed, trying JSON lines. Error:", e)
            sample = []
            for i, line in enumerate(text.splitlines()):
                if line.strip():
                    try:
                        sample.append(json.loads(line))
                    except Exception as line_e:
                        with output_box:
                            print(f"Line {i+1} failed: {line_e}")
            if not sample:
                with output_box:
                    print("❌ No valid JSON objects found in file.")
                return
        sample_obj = sample[0] if isinstance(sample, list) else sample
    except Exception as e:
        with output_box:
            print(f"❌ Failed to parse sample file: {e}")
        return

    supplier_id = supplier_input.value.strip()
    count = count_input.value

    if not supplier_id:
        with output_box:
            print("❌ SupplierID is required.")
        return

    generated = generate_data(sample_obj, count, supplier_id)

    with output_box:
        print(f"✅ Generated {count} records with SupplierID: {supplier_id}")
        display(generated[:3])  # Show a sample preview

        filename = f"generated_{supplier_id}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        filepath = os.path.join(os.getcwd(), filename)

        with open(filepath, "w") as f:
            json.dump(generated, f, indent=4)

        with output_box:
            print(f"📁 File saved as: {filepath}")

# Attach the event handler to the button
generate_btn.on_click(on_generate_clicked)

### Display Widgets

In [ ]:
display(widgets.HTML("<b>Enter SupplierID and Number of Records:</b>"))
display(supplier_input, count_input, generate_btn, output_box)